In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import pandas as pd
import time
import os
from numpy import random
import pickle
# from playsound import playsound

from bs4 import BeautifulSoup

In [2]:
def flatten(t):
    return [item for sublist in t for item in sublist]

In [18]:
chrome_options = webdriver.ChromeOptions()
prefs = {"profile.managed_default_content_settings.images": 2}
chrome_options.add_experimental_option("prefs", prefs)

In [19]:
if os.name == 'nt':
    driver = webdriver.Chrome('C:/Users/user/Documents/chromedriver.exe')
else:
    driver = webdriver.Chrome('/home/guillermo/Documentos/chromedriver_linux64/chromedriver', options=chrome_options)

In [5]:
urls= ['https://www.gallito.com.uy/inmuebles/casas/venta/maldonado/punta-del-este/ord_asc?pag={}'.format(i) for i in range(1, 3)]

In [6]:
data = []
url_publicacion = []

for url in urls:
    try:
        driver.get(url)
    except:
        print('Get method failed, check URL.')
    pagina = [e.text.splitlines() for e in driver.find_elements_by_css_selector('.contenedor-info')]
    data.append(pagina)
    html = [e.get_attribute("innerHTML") for e in driver.find_elements_by_css_selector('.contenedor-info .mas-info')]
    for i in html:
        soup = BeautifulSoup(i, "html.parser")
        url_publicacion.append(soup.find('a')['href'])
flat_list = flatten(data)
result = pd.DataFrame(flat_list, columns=['desc', 'valor'])

result['url'] = url_publicacion

In [7]:
result

,desc,valor,url
0,3 Dormitorios - 145 m2,U$S 1,https://www.gallito.com.uy/venta-casa-3-dorm-t...
1,4 Dormitorios - 510 m2,U$S 798,https://www.gallito.com.uy/hermosa-casa-en-fra...
2,3 Dormitorios - 207 m2,U$S 5.200,https://www.gallito.com.uy/casa-en-playa-mansa...
3,4 Dormitorios,U$S 7.900,https://www.gallito.com.uy/casa-4-dormitorios-...
4,1 Ambiente - 40 m2,U$S 39.000,https://www.gallito.com.uy/venta-casa-la-barra...
...,...,...,...
75,3 Dormitorios,U$S 175.000,https://www.gallito.com.uy/casa-en-un-en-entor...
76,3 Dormitorios,U$S 175.000,https://www.gallito.com.uy/casa-en-mansa-3-dor...
77,3 Dormitorios,U$S 175.000,https://www.gallito.com.uy/muy-linda-casa-en-v...
78,3 Dormitorios,U$S 175.000,https://www.gallito.com.uy/oportunidad-rincon-...


In [ ]:
# recupera latlong si hay
coordenadas = []

driver.get('https://www.gallito.com.uy/casa-en-venta-la-barra-1-dormitorio-inmuebles-17750354')

elem = driver.find_elements_by_css_selector('#ubicacion')

if elem != []:
    element = driver.find_element_by_xpath("/html/body/form/main/div/div[1]/ul/li[4]/a/i")
    element.click()
    time.sleep(2)
    ubic_html = [e.get_attribute("innerHTML") for e in driver.find_elements_by_css_selector("#ubicacion")]
    soup = BeautifulSoup(ubic_html[0], "html.parser")
    src = soup.find('iframe')['src']
    latlng = src.split('q=', 1)[1].split('&zoom=', 1)[0]
    coordenadas.append(latlng)
else:
    print('Sin ubicación')
    coordenadas.append('')


In [ ]:
coordenadas

In [ ]:
coordenadas = []

for url in urls:
    driver.get(url)

    elem = driver.find_elements_by_css_selector('#ubicacion')

    if elem != []:
        element = driver.find_element_by_xpath("/html/body/form/main/div/div[1]/ul/li[4]/a/i")
        element.click()
        time.sleep(2)
        ubic_html = [e.get_attribute("innerHTML") for e in driver.find_elements_by_css_selector("#ubicacion")]
        soup = BeautifulSoup(ubic_html[0], "html.parser")
        src = soup.find('iframe')['src']
        latlng = src.split('q=', 1)[1].split('&zoom=', 1)[0]
        coordenadas.append(latlng)
    else:
        print('Sin ubicación')
        coordenadas.append('')

In [7]:
def scrapper(urls):
    data = []
    metraje = []
    url_publicacion=[]

    for url in urls:
        try:
            driver.get(url)
            time.sleep(random.uniform(0.9, 2.3))
        except:
            print('Get method failed, check URL.')
        pagina = [e.text.splitlines() for e in driver.find_elements_by_css_selector('.contenedor-info')]
        data.append(pagina)
        html = [e.get_attribute("innerHTML") for e in driver.find_elements_by_css_selector('.contenedor-info .mas-info')]

        for i in html:
            soup = BeautifulSoup(i, "html.parser")
            url_publicacion.append(soup.find('a')['href'])

    flat_list = flatten(data)
    result = pd.DataFrame(flat_list, columns=['desc', 'valor'])
    result['url'] = url_publicacion

    return result

In [8]:
urls= ['https://www.gallito.com.uy/inmuebles/casas/venta/maldonado/punta-del-este/ord_asc?pag={}'.format(i) for i in range(1, 97)]

In [9]:
start = time.time()

data = scrapper(urls)

end = time.time()

Get method failed, check URL.


ProtocolError: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))

In [22]:
(end-start)/60

7.052405750751495

In [19]:
data.shape

(3832, 3)

In [20]:
data.to_csv('ventas_casa_punta_del_este_16072021.csv', index=False)

## Recupera coordenadas

In [13]:
data = pd.read_csv('ventas_casa_punta_del_este_16072021.csv')
data.head()

,desc,valor,url,lat,lng
0,3 Dormitorios - 145 m2,U$S 1,https://www.gallito.com.uy/venta-casa-3-dorm-t...,-34.92605,-54.89400
1,4 Dormitorios - 510 m2,U$S 798,https://www.gallito.com.uy/hermosa-casa-en-fra...,-34.90977,-55.02943
2,3 Dormitorios - 207 m2,U$S 5.200,https://www.gallito.com.uy/casa-en-playa-mansa...,-34.94080,-54.94299
3,4 Dormitorios,U$S 7.900,https://www.gallito.com.uy/casa-4-dormitorios-...,-34.93705,-54.94775
4,1 Ambiente - 40 m2,U$S 39.000,https://www.gallito.com.uy/venta-casa-la-barra...,-34.89949,-54.81126


In [14]:
def scrap_latlng(urls):
    coordenadas = []

    for url in urls:
        try:
            driver.get(url)
            time.sleep(1)
        except:
            print('Get method failed, check URLs.')

        elem = driver.find_elements_by_css_selector('#ubicacion')

        if elem != []:
            element = driver.find_element_by_xpath("/html/body/form/main/div/div[1]/ul/li[4]/a/i")
            element.click()
            time.sleep(1)
            ubic_html = [e.get_attribute("innerHTML") for e in driver.find_elements_by_css_selector("#ubicacion")]
            soup = BeautifulSoup(ubic_html[0], "html.parser")
            src = soup.find('iframe')['src']
            latlng = src.split('q=', 1)[1].split('&zoom=', 1)[0]
            if latlng == '/,/':
                coordenadas.append('Nan,Nan')
            else:
                coordenadas.append(latlng)
        else:
            coordenadas.append('Nan,Nan')

    return coordenadas

In [15]:
urls = list(data.url.values)
len(urls)

3832

In [16]:
urls_1 = urls[0:1000]
urls_2 = urls[1000:2000]
urls_3 = urls[2000:3000]
urls_4 = urls[3000:4000]

len(urls_4)

832

In [20]:
# CORRE EL SCRAPPER
coords_1 = scrap_latlng(urls_1)

# # guarda en in pickle
# with open('coords_1.pkl', 'wb') as f:
#     pickle.dump(coords_1, f)

Get method failed, check URLs.


ProtocolError: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))

In [34]:
coords_2 = scrap_latlng(urls_2)

with open('coords_2.pkl', 'wb') as f:
    pickle.dump(coords_2, f)

In [35]:
coords_3 = scrap_latlng(urls_3)

with open('coords_3.pkl', 'wb') as f:
    pickle.dump(coords_3, f)

In [36]:
coords_4 = scrap_latlng(urls_4)

with open('coords_4.pkl', 'wb') as f:
    pickle.dump(coords_4, f)

### Levanta las coodenadas

In [37]:
data = pd.read_csv('ventas_casa_punta_del_este_16072021.csv')
data.head()

,desc,valor,url
0,3 Dormitorios - 145 m2,U$S 1,https://www.gallito.com.uy/venta-casa-3-dorm-t...
1,4 Dormitorios - 510 m2,U$S 798,https://www.gallito.com.uy/hermosa-casa-en-fra...
2,3 Dormitorios - 207 m2,U$S 5.200,https://www.gallito.com.uy/casa-en-playa-mansa...
3,4 Dormitorios,U$S 7.900,https://www.gallito.com.uy/casa-4-dormitorios-...
4,1 Ambiente - 40 m2,U$S 39.000,https://www.gallito.com.uy/venta-casa-la-barra...


In [42]:
arc = ['coords_{}.pkl'.format(i) for i in range(1,5)]

coords = pd.read_pickle(arc[0]) + pd.read_pickle(arc[1]) + pd.read_pickle(arc[2]) + pd.read_pickle(arc[3])

len(coords)

3832

In [43]:
data[['lat', 'lng']] = [i.split(',') for i in coords]

In [44]:
data['lat'] = data['lat'].astype(float).round(5)
data['lng'] = data['lng'].astype(float).round(5)

In [46]:
data.head(18)

,desc,valor,url,lat,lng
0,3 Dormitorios - 145 m2,U$S 1,https://www.gallito.com.uy/venta-casa-3-dorm-t...,-34.92605,-54.89400
1,4 Dormitorios - 510 m2,U$S 798,https://www.gallito.com.uy/hermosa-casa-en-fra...,-34.90977,-55.02943
2,3 Dormitorios - 207 m2,U$S 5.200,https://www.gallito.com.uy/casa-en-playa-mansa...,-34.94080,-54.94299
3,4 Dormitorios,U$S 7.900,https://www.gallito.com.uy/casa-4-dormitorios-...,-34.93705,-54.94775
4,1 Ambiente - 40 m2,U$S 39.000,https://www.gallito.com.uy/venta-casa-la-barra...,-34.89949,-54.81126
5,1 Dormitorio - 1139 m2,U$S 74.000,https://www.gallito.com.uy/casa-punta-del-este...,-34.90194,-54.86797
6,1 Ambiente - 32 m2,U$S 85.000,https://www.gallito.com.uy/casa-en-venta-inmue...,-34.92777,-54.95606
7,2 Dormitorios - 80 m2,U$S 90.000,https://www.gallito.com.uy/casa-en-punta-del-e...,-34.87209,-55.09042
8,1 Dormitorio,U$S 90.000,https://www.gallito.com.uy/casa-en-la-barra-1-...,NaN,NaN
9,2 Dormitorios,U$S 92.000,https://www.gallito.com.uy/casa-en-balneario-b...,NaN,NaN


In [47]:
data.to_csv('ventas_casa_punta_del_este_16072021.csv', index=False)

In [ ]:
# limpieza
data['valor'].str.replace('U', '')

for word, rep in {"U":" ", "S":"", "$":"", ".":""}.items():
    data['valor'] = data['valor'].str.replace(word, rep, regex=False)

In [ ]:
data['valor'] = data['valor'].astype(int)

In [ ]:
len(data)

In [ ]:
data = data.loc[~((data.valor == 111111111) | (data.valor < 45000))]
data.head()

In [ ]:
data[['dormitorios', 'barrio']] = data['desc'].str.split(' en ', n = 1, expand = True)

In [ ]:
data['barrio_ine'] = data['barrio']

In [ ]:
def format_barrio(df, column):
    "Formatea strings de Barrio Gallito a Barrio INE"
    barrios_dict = {
        'Aguada' : 'Aguada',
        'Aires Puros' : 'Aires Puros',
        'Atahualpa' : 'Atahualpa',
        'B. De Carrasco' : 'Bañados de Carrasco',
        'Barrio Sur' : 'Barrio Sur',
        'Belvedere' : 'Belvedere',
        'Brazo Oriental' : 'Brazo Oriental',
        'Buceo' : 'Buceo',
        'Capurro' : 'Capurro, Bella Vista',
        'Carrasco' : 'Carrasco',
        'Carrasco Norte' : 'Carrasco Norte',
        'Casabo' : 'Casabó, Pajas Blancas',
        'Casavalle' : 'Casavalle',
        'Centro' : 'Centro',
        'Cerrito' : 'Cerrito',
        'Cerro' : 'Cerro',
        'Ciudad Vieja' : 'Ciudad Vieja',
        'Cno. Maldonado' : 'Colón Centro y Noroeste',
        'Colon' : 'Colón Sureste, Abayubá',
        'Cordon' : 'Cordón',
        'Golf' : 'Flor de Maroñas',
        'Ituzaingo' : 'Ituzaingó',
        'J. Hipodromo' : 'Jardines del Hipódromo',
        'Jacinto Vera' : 'Jacinto Vera',
        'La Blanqueada' : 'La Blanqueada',
        'La Comercial' : 'La Comercial',
        'La Figurita' : 'La Figurita',
        'La Teja' : 'La Teja',
        'Larrañaga' : 'Larrañaga',
        'Las Acacias' : 'Las Acacias',
        'Lezica' : 'Lezica, Melilla',
        'Malvin' : 'Malvín',
        'Malvin Norte' : 'Malvín Norte',
        'Manga' : 'Manga',
        'Maroñas' : 'Maroñas, Parque Guaraní',
        'Maroñas Curva' : 'Maroñas, Parque Guaraní',
        'Melilla' : 'Lezica, Melilla',
        'Nuevo Paris' : 'Mercado Modelo, Bolívar',
        'Pajas Blancas' : 'Nuevo París',
        'Palermo' : 'Palermo',
        'Parque Batlle' : 'Parque Batlle, Villa Dolores',
        'Parque Rodo' : 'Parque Rodó',
        'Paso De La Arena' : 'Paso de la Arena',
        'Peñarol' : 'Peñarol, Lavalleja',
        'Perez Castellanos' : 'Castro, P. Castellanos',
        'Piedras Blancas' : 'Piedras Blancas',
        'Pocitos' : 'Pocitos',
        'Pocitos Nuevo' : 'Pocitos',
        'Prado' : 'Prado, Nueva Savona',
        'Prado Norte' : 'Prado, Nueva Savona',
        'Puerto Buceo' : 'Buceo',
        'Punta Carretas' : 'Punta Carretas',
        'Punta Gorda' : 'Punta Gorda',
        'Punta Rieles' : 'Punta Rieles, Bella Italia',
        'Reducto' : 'Reducto',
        'Sayago' : 'Sayago',
        'Tres Cruces' : 'Tres Cruces',
        'Union' : 'Unión',
        'Villa Dolores' : 'Parque Batlle, Villa Dolores',
        'Villa Española' : 'Villa Española',
        'Villa Muñoz' : 'Villa Muñoz, Retiro'
        }

    return df[column].map(barrios_dict)

In [ ]:
data['barrio_ine'] = format_barrio(data, 'barrio')

In [ ]:
data.head(15)

In [ ]:
def encode_barrio(df, column):
    "Codifica barrio INE"
    barrios_dict = {
    'Ciudad Vieja':1,
    'Centro':2,
    'Barrio Sur':3,
    'Cordón':4,
    'Palermo':5,
    'Parque Rodó':6,
    'Punta Carretas':7,
    'Pocitos':8,
    'Buceo':9,
    'Parque Batlle, Villa Dolores':10,
    'Malvín':11,
    'Malvín Norte':12,
    'Punta Gorda':13,
    'Carrasco':14,
    'Carrasco Norte':15,
    'Bañados de Carrasco':16,
    'Maroñas, Parque Guaraní':17,
    'Flor de Maroñas':18,
    'Las Canteras':19,
    'Punta Rieles, Bella Italia':20,
    'Jardines del Hipódromo':21,
    'Ituzaingó':22,
    'Unión':23,
    'Villa Española':24,
    'Mercado Modelo, Bolívar':25,
    'Castro, P. Castellanos':26,
    'Cerrito':27,
    'Las Acacias':28,
    'Aires Puros':29,
    'Casavalle':30,
    'Piedras Blancas':31,
    'Manga, Toledo Chico':32,
    'Paso de las Duranas':33,
    'Peñarol, Lavalleja':34,
    'Cerro':35,
    'Casabó, Pajas Blancas':36,
    'La Paloma, Tomkinson':37,
    'La Teja':38,
    'Prado, Nueva Savona':39,
    'Capurro, Bella Vista':40,
    'Aguada':41,
    'Reducto':42,
    'Atahualpa':43,
    'Jacinto Vera':44,
    'La Figurita':45,
    'Larrañaga':46,
    'La Blanqueada':47,
    'Villa Muñoz, Retiro':48,
    'La Comercial':49,
    'Tres Cruces':50,
    'Brazo Oriental':51,
    'Sayago':52,
    'Conciliación':53,
    'Belvedere':54,
    'Nuevo París':55,
    'Tres Ombúes, Victoria':56,
    'Paso de la Arena':57,
    'Colón Sureste, Abayubá':58,
    'Colón Centro y Noroeste':59,
    'Lezica, Melilla':60,
    'Villa García, Manga Rural':61,
    'Manga':62
        }
    return df[column].map(barrios_dict)

In [ ]:
data['cod_barrio_ine'] = encode_barrio(data, 'barrio_ine')
data.head()

In [ ]:
data.to_csv('ventas_mvdeo_20210707_depurado.csv', index=False)

In [ ]:
data = data.loc[(data.metraje > 10) & (data.metraje < 4000) & (~data.metraje.isna())]

data['metraje'] = data['metraje'].astype(int)

data['valor_metro'] = data['valor'] / data['metraje']

agru = data.groupby('cod_barrio_ine').median('valor_metro').reset_index()

agru['cod_barrio_ine'] = agru['cod_barrio_ine'].astype(int)
                                                       
agru.to_csv('agru.csv', index=False)